<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/hungarian/python/effective_algoritm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esettanulmány

## Bevezetés

A prím számok kikeresésére szolgáló algoritmus példáján a hatékony algoritmus kialakítását és a Pythonic kód készítését mutatjuk be.

## Első naiv algoritmus

Prím szám az a természtes szám melynek két osztója van (önmaga és egy). A legkisebb prím szám a kettő. Egy számról úgy dönthetjük el prím-e, hogy végig próbáljuk a kisebb számokkal mennyi lesz az osztási maradékuk. El kell menni az oszthatóság vizsgálatával n-1-ig ha n a vizsgált szám? A szám gyökénél nagyobb számokra nem érdemes vizsgálnunk hiszen például a 24 esetén a négyes osztó megtalálása után nincs jelentősége, hogy négyeshez tartozó osztópárt (6) is megtaláljuk. Ez Python-ban így nézhet ki:

In [ ]:
max_num = 500001    # largest number + 1 to search primes
import math
import time

In [ ]:
"""
    naive algorith to find prime numbers
    version 1.0
"""

start_time = time.time()
prims = []                   # list of prims
for p in range(2, max_num):   # find prims up to max_num
    prime = True
    for divider in range(2, int(math.sqrt(p))+1):
        if p % divider == 0:     # remainder of division is zero
            prime = False        # it is not a prime
    if prime:
        prims.append(p)      # store prime number
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 33.474411 seconds


Az algoritmus hatékonyságát az algoritmusunk futási idejével mérjük. A mai számítógépeken mindig több alkalmazás, szolgáltatás fut párhuzamosan, ezért az egyszeri időmérés nem ad átlagos eredményt, Célszerű többször futtatni az átlagos futási idő megtalálásához.

## Első hatékonyságnövelés

A fenti algoritmus 105 esetén 11-ig tart az osztók vizsgálata, azonban a 3-as osztó megtalálása után felesleges tovább folytatni a belső ciklust, már eldőlt nem prím számról van szó. Módosítsuk az algoritmus, hogy az első osztó megtalálása után a belső ciklusból lépjen ki (break utasítás).

Ha megnézzük a vizsgált számokra (*p*-re) vonatkozó ciklust, rájöhetünk, hogy a kettőnél nagyobb számok esetén felesleges a páros számokat vizsgálni. Módosítsuk a ciklust, hogy háromtól csak a páratlan számokat ellenőrizzük. Feltételezzük, hogy a *max_num* nagyobb kettőnél.

In [ ]:
"""
    naive algorith to find prime numbers
    version 1.1
"""

start_time = time.time()
prims = [2]                   # list of prims
for p in range(3, max_num, 2):   # find prims up to 50000
    prime = True
    for divider in range(2, int(math.sqrt(p))+1):
        if p % divider == 0: # remainder of division is zero
            prime = False    # it is not a prime
            break            # divider found no need to continue
    if prime:
        prims.append(p)      # store prime number
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 3.297944 seconds


Az első változat közel 30 másodpercig fut. A második változat tízszer gyorsabb. Az egymásba ágyazott ciklusok esetén a belső ciklus futásának a lerövidítése nagy hatékonyság növekedéssel jár.

## Tegyük Pythonikussá a kódot

A bevezetőben említettük, hogy nem csak a hatékonyság, hanem a Pythonikus (Pythonic) kód kialakítása is a célunk. A Python nyelvben a ciklushoz is rendelhetünk egy else utasítást, mely akkor hajtódik végre, ha nem léptünk ki a ciklus futtatásából break utsítással. Ennek felhasználásával rövidebbé tehetjük a kódunkat és ezzel talán könnyebben olvashatóvá. Feleslegessé válik a prím logikai változó használata.

In [ ]:
"""
    naive algorith to find prime numbers
    version 1.2
"""

start_time = time.time()
prims = [2]                   # list of prims
for p in range(3, max_num, 2):  # find prims up to max_num
    for divider in range(2, int(math.sqrt(p))+1):
        if p % divider == 0: # remainder of division is zero
            break            # divider found no need to continue
    else:
        prims.append(p)      # store prime number
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 3.282509 seconds


Ezzel a módosítással a kódunk nem vált hatékonyabbá, de a kevesebb utasításból álló kód előnyösebb.

Minden nem prím szám felbontható prím számok szorzatára. Így az oszthatóság vizsgálatot elég az előzőleg megtalált prím számokra végrehajtani. Módosítsuk az algoritmusunkat.

In [ ]:
"""
        naive algorith to find prime numbers
        version 1.3
"""

start_time = time.time()
prims = [2]                   # list of prims
for p in range(3, max_num, 2):  # find prims up to max_num
        maxp = int(math.sqrt(p))+1
        for divider in prims:    # enough to check prims!
                if p % divider == 0: # remainder of division is zero
                        break            # divider found no need to continue
                if maxp < divider:
                        prims.append(p)
                        break
        else:
                prims.append(p)      # store prime number
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 1.207073 seconds


## Hatékonyabb algoritmus

Az előzőekben az eredeti elképzelésünket megtartva módosítottuk a kódot a hatékonyság érdekében. Lehet, hogy az eredeti elképzelésünk átértékelésével juthatunk hatékonyabb megoldáshoz? Ez már Eraszthotenésznek is sikerült az eraszthotenészi szita kitalálásával. Ennek alapgondolata, hogy ne az egyes vizsgált számok osztásával keressük a prímeket, hanem állítsuk elő a természetes számok sorozatát és ebből távolítsuk el az egyes számok többszöröseit. Ez valahogy így nézhet ki:

In [ ]:
"""
    Sieve of Erasthotenes prim algorithm
    version 2.0
"""

start_time = time.time()
numbers = list(range(max_num))     # list of natural numbers to check
for j in range(2, int(math.sqrt(max_num))):
    numbers[j+j::j] = [0 for k in numbers[j+j::j]] # use sieve

prims = sorted(list(set(numbers) - set([0, 1]))) # remove zeros from list
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 0.212563 seconds


A hatékonyságnövekedés jól látható. A kódban a listaértelmezést (list comprehension) alkalmaztuk. Ez gyorsabb mint a lista *for* típusú ciklussal előállítása. A

`[0 for k in numbers[j+j::j]]`

sor egy nullákat tartalmazó listát állít elő, melynek a hossza megfelel a j érték többszörösei számának. Az értékadással a számok listájában nullázzuk a j érték többszöröseit. Nem lehetett volna egyszerűen a következő értékadást írni?

`numbers[j+j::j] = 0`

Sajnos ez nem működik, egy lista részének nem adhatunk értékül egy skalárt, de a [0] sem működik az értékadás jobb oldalán, mert az is csak folytonos részére működne az eredeti listának és a megadott tartományt egy 0 elemre cserélné.

Ez a változat fél millióig a prím számokat 3 tized másodperc alatt állítja elő. Az első algoritmusunkhoz képest százszoros gyorsulást értünk el.

## Lehet még gyorsítani?

Elemezzük egy kicsit a kódunkat. A j ciklusváltozó a 2, 3, 4, ... értékeket veszi fel a futás során, így először 4-től nullázzuk az összes páros számot, majd 6-tól minden harmadik számot, majd 8-tól minden negyediket. Álljunk meg itt egy pillanatra! Minek nullázzuk a néggyel osztható számokat? Azokat már a kettővel oszthatóság miatt nulláztuk. Hasonló a helyzet például a kilenccel osztható számokkal, azokat már a hárommal oszthatóság miatt nulláztuk. Azaz nem kell minden `j`-re az elemek nullázását végrehajtani, erre csak akkor van szükség, ha `j`-ik elemet még nem nulláztuk (azaz prím). Ez egy plusz feltétellel tehetjük meg, mellyel a kód hosszabb lesz, de hatékonyabb.

In [ ]:
"""
    Sieve of Erasthotenes prim algorithm
    version 2.1
"""

start_time = time.time()
numbers = list(range(max_num))     # list of natural numbers to check
for j in range(2, int(math.sqrt(max_num))):
    if numbers[j]:
        numbers[j+j::j] = [0 for k in numbers[j+j::j]] # use sieve

prims = sorted(list(set(numbers) - set([0, 1]))) # remove zeros from list
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 0.141774 seconds


Ennek a módosításnak a hatékonyság növelő hatása fél millióig futtatva kevésbé jelentkezik. Ennek az is az oka, hogy az algoritmusunk futási ideje maximális prím szám növelésével nem lineárisan növekszik.

A lista értelmezés hatékonyabb módszer a listák előállítására mint a "sima" `for` ciklus. Azonban az esetünkben az előállított lista minden eleme nulla. A lista értelmezést arra használjuk, hogy a lista hosszát be tudjuk állítani. Erre viszont létezik egy egyszerűbb (pythonikusabb) megoldás. Ha egy listát egy egész számmal szorzunk, akkor az eredmény a lista többszörözése. A

`[0] * 5`

utasítás egy öt hosszúságú nullákat tartalmazó listát eredményez. Nézzük meg, hogy egy ilyen átalakítás növeli-e a hatékonyságot!

In [ ]:
"""
    Sieve of Erasthotenes prim algorithm
    version 2.2
"""

start_time = time.time()
numbers = list(range(max_num))     # list of natural numbers to check
for j in range(2, int(math.sqrt(max_num))):
    if numbers[j]:
        numbers[j+j::j] = [0] * len(numbers[j+j::j]) # use sieve
prims = sorted(list(set(numbers) - set([0, 1]))) # remove zeros from list
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))


ready
41538 prims in 0.091630 seconds


## A numpy könyvtár még egy kicsit gyorsíthat

A `numpy` Python modul számos matemetikai probléma megoldásába kész megoldásokat nyújt a programozóknak. Mi a `numpy` tömb kezelését és több tömb elemre érték adást használjuk fel.

In [ ]:
import numpy as np

In [ ]:
"""
    Sieve of Erasthotenes prim algorithm
    version 2.3
"""

start_time = time.time()
numbers = np.arange(max_num)     # list of natural numbers to check
for j in range(2, int(math.sqrt(max_num))):
    if numbers[j]:
        numbers[j+j::j] = 0 # use sieve
prims = sorted(list(set(numbers) - set([0, 1]))) # remove zeros from list
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 0.061418 seconds


A `numpy` modul importlásán kívül csak két sor módosult. A számok előálltása során egy `numpy` tömböt hozunk létre az `arange` függvénnyel. A gyorsítást a második módosítás jelenti, az elemek nullázásához nem kell előállítanunk annyi nulla elemből álló listát, ahány elemet nullázni szeretnénk. Ezzel további gyorsulást érhetünk el, persze itt ebbe nem számítottukbe a `numpy` modul betöltésének idejét.

Az egyes algoritmusokat a 100000-nél, 1000000-nál és 10000000-nál kisebb prím számok kikeresére futtattuk. Az alábbi táblázat tartalmazza a futási időket másodpercben:

| Verzió | 100 000 | 1 000 000 | 10 000 000 |
|--------|---------|-----------|------------|
|    1.0 |    1.90 |        60 |  > 9999999 |
|    1.1 |    0.45 |        10 |        326 |
|    1.2 |    0.44 |        11 |        333 |
|    1.3 |    0.21 |      2.62 |         50 |
|    2.0 |    0.07 |      0.58 |       6.41 |
|    2.1 |    0.04 |      0.32 |       2.99 |
|    2.2 |    0.02 |      0.19 |       1.73 |
|    2.3 |    0.03 |      0.17 |       1.61 |

##Wilson tétel

Wilson prímekre vonatkozó tétele a következő: *n* > 1 természetes szám prím akkor és csak akkor, ha a számnál kisebb pozitív egész számok szorzata eggyel kisebb mint *n* egy többszöröse.

$(n - 1)! + 1) \% n = 0$

In [ ]:
"""
    Wilson's theorem prim algorithm
    version 3.0
"""

start_time = time.time()
max_num = 10001
numbers = range(max_num)     # list of natural numbers to check

def is_prime(j):
    return j == 2 or (j > 1 and j % 2 != 0 and (math.factorial(j-1) + 1) % j == 0)

primes = [x for x in numbers if is_prime(x)]
print(len(primes), time.time() - start_time)

1229 6.25184965133667


A fenti megoldás jóval kevésbé hatékony az erasztothenészi szitánál. A faktoriális számítás időigényes, de ha tudjuk, hogy $n! = n * (n-1)!$. Így a következő faktoriális számítását egy szorzással megoldhatjuk a *math.factorial* függvény hívása nélkül.

In [ ]:
"""
    Wilson's theorem prim algorithm
    version 3.1
"""

start_time = time.time()
max_num = 10001
numbers = range(2, max_num)     # list of natural numbers to check
fact = 1                        # first factorial
primes = []
for j in numbers:
    if (fact + 1) % j == 0:
        primes.append(j)
    fact *= j
print(len(primes), time.time() - start_time)

1229 0.32576799392700195


Gyorsíthatunk még a megoldásunkon, ha kettőnél nagyobb páros számokat kihagyjuk a vizsgálatból.

In [ ]:
"""
    Wilson's theorem prim algorithm
    version 3.2
"""

start_time = time.time()
max_num = 10001
numbers = range(3, max_num, 2)     # list of natural numbers to check
fact = 1                           # first factorial
primes = [2]
for j in numbers:
    if (fact * (j-1) + 1) % j == 0:
        primes.append(j)
    fact *= j * (j-1)
print(len(primes), time.time() - start_time)

1229 0.20883750915527344


## Következtetések

Három különböző algoritmust vizsgáltunk meg a a prim számok keresésére. A legnagyobb hatékonyságot az eraszthotenészi szitával értük el. A feladat megoldásában a hatékonyság növelést egyrészt a hatékonyabb algoritmus kiválasztásával, másrészt az algoritmus finomításával, harmadrészt pedig a jobb programozási megoldásokkal értük el.

Nem biztos, hogy minden esetben az eraszthotenészi szita a legjobb megoldás, például, ha a számítógépünkben több mag található, akkor a párhuzamos programozás egy további gyorsítási lehetőséget ad. Viszont problémát jelenthet, hogy nem minden algoritmus alakítható át párhuzamosan futtathatóvá. Esetünkben az első algoritmusunk az 1.2 és a harmadik algoritmusunk 3.0 verziója alkalmas a párhuzamosításra. Minden egyes vizsgált számra párhuzamosan futtatható annak az eldöntése, hogy prim szám-e. A párhuzamosítás történhet a CPU magjainak felhasználásával vagy a GPU magjainak a felhasználásával. Amíg a CPU-knak 1-24 közötti magja van addig a GPU-k ezret elérő maggal rendelkezhetnek. Például mesterséges intelligencia alkalmazások a mesterséges neurális hálózatok tanítása során jellemző a GPU magok felhasználása a számítások párhuzamosítása során.



##Feladatok

Készítsen grafikont a keresési számtartomány futási idő szerinti függvényéről a különböző verziók esetén.